### Ask a LLM about board game rules, using the board game rulebook  


Here we implement a simple RAG (Retrieval Augmented Generation) system in order to ask simple questions answered based on the game rulebook.  
Because we had to choose a very small model, we could not provide too much context to the LLM.  
During tests where we provided too much content, the LLM got confused and produced serious hallucinations. 
So, you can ask questions that can be answered in 300-600 words in the rulebook.  
It's better to avoid more complex questions like "For game X, please tell me how to setup a 2-player game".  
However, if you want to try more complex questions, play with the variables `number_of_results` and `extend_results_by`.  
Have in mind that larger values need more VRAM and increase execution time exponentially.  

Embedding model chosen from:  
https://huggingface.co/spaces/mteb/leaderboard  
We need a small but capable model.

In [13]:
import torch
import chromadb

In [2]:
from customEmbedFunction import TransformerEmbeddingFunction

In [3]:
chroma_client = chromadb.PersistentClient(path="chroma")
embedder_name = "w601sxs/b1ade-embed"
collection = chroma_client.get_or_create_collection(name='bg-rag', embedding_function=TransformerEmbeddingFunction(embedder_name, use_cuda=True))

During the tests, the name of the game carried a lot of semantic meaning for the returned results.  
It was 'masking' the semantic meaning of the actual question.  
As a solution, first we get a sample of results to find the game name.  
Then we select new results just for this game and replace the name of the game in the question text with the generic 'the game'.  

In [4]:
questions = []
questions.append("For The Witcher: Old World, please tell me how to setup a 2-player game.")
questions.append("In The Witcher: Old World, can I play poker and fight a player at the same turn?")
questions.append("In Dune: Imperium - Uprising, please explain in detail how spies and observation posts work.")
number_of_results = 2 # This should be the perfect value for this model.
extend_results_by = 1 # For complex questions that need larger context, choose 2.
selected_question = 0 # Choose the question you want to try by changing the index.

question = questions[selected_question]

# Find the game of the question
results = collection.query(
    query_texts=[question], 
    n_results=3, 
    )

# print(results['ids'])

# Get the name of the game (the most common value)
games = [result['bg_name'] for result in results['metadatas'][0]]
game_name = max(set(games), key=games.count)

question = question.replace(game_name, "the game", -1)

# Get results for the specific game
results = collection.query(
    query_texts=[question], 
    n_results=number_of_results, 
    where={"bg_name": game_name},
    )

c:\Users\eltar\anaconda3\envs\RAG-YT-History\Lib\site-packages\transformers\models\bert\modeling_bert.py:435: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [ ]:
results['ids']

[['TWOW_07_Rulebook_Main_(36p)_EN_preview_22-10-20-skompresowany.md_09',
  'TWOW_07_Rulebook_Main_(36p)_EN_preview_22-10-20-skompresowany.md_10',
  'TWOW_07_Rulebook_Main_(36p)_EN_preview_22-10-20-skompresowany.md_42']]

In [6]:
results['documents']

[['| | | |8.2| | | | | |10.3| | | | |10.2| | | | |10.1| | | |9| |6| Setup for a 2-Player Game 9 Starting Player and Help Cards The Player who most recently read a Witcher book is the Starting Player; alternatively, you can determine this randomly. Each Player takes an Action and Fight Help Card. Clarification: The Starting Player of the game should be making the last choice (in a 5-Player game the Starting Player draws the last remaining Player Board). 10 Player Components Each Player does the following: 1. Starting with the Player to the right of the starting Player, then continuing in counter-clockwise order, each Player takes a Player Board, by performing the following steps (one at a time): - Shuffle all (unchosen) Player Boards face-down. - Draw 2; then, choose 1 to keep. (Return the unchosen one back with the remaining Boards.) Optionally: Players can distribute Player Boards using any method they choose. 1. Take the Scoring Token (matching your School color) and place it on the 

Semantic search cannot get the full related context, only the most relevant chunks.  
As a solution, we select surrounding chunks (previous and next chunks for each chunk) to try and get the full context.  

In [7]:
# Select surrounding documents/chunks (previous and next for each document), in case we missed some content.
ids = results['ids'][0]

# Extracting the numbers from the ids and converting to integers
numbers = [int(id.split('_')[-1]) for id in ids]

# Getting the surrounding ids (chunks) based on the offset
offset = extend_results_by
result_list = []
for id in ids:
    num = int(id.split('_')[-1])
    filename = id.rsplit('_', 1)[0]
    for i in range(-offset, offset + 1):
        surrounding_id = f'{filename}_{num + i:02d}'
        result_list.append(surrounding_id)

result_list = list(dict.fromkeys(result_list))

print(result_list)

['TWOW_07_Rulebook_Main_(36p)_EN_preview_22-10-20-skompresowany.md_08', 'TWOW_07_Rulebook_Main_(36p)_EN_preview_22-10-20-skompresowany.md_09', 'TWOW_07_Rulebook_Main_(36p)_EN_preview_22-10-20-skompresowany.md_10', 'TWOW_07_Rulebook_Main_(36p)_EN_preview_22-10-20-skompresowany.md_11', 'TWOW_07_Rulebook_Main_(36p)_EN_preview_22-10-20-skompresowany.md_41', 'TWOW_07_Rulebook_Main_(36p)_EN_preview_22-10-20-skompresowany.md_42', 'TWOW_07_Rulebook_Main_(36p)_EN_preview_22-10-20-skompresowany.md_43']


In [8]:
# Get the extended results.
# Unfortunately, these come back sorted by id, so our semantically relevant chunks may be in the wrong order.
results = collection.get(ids=result_list)

In [9]:
results['ids']

['TWOW_07_Rulebook_Main_(36p)_EN_preview_22-10-20-skompresowany.md_08',
 'TWOW_07_Rulebook_Main_(36p)_EN_preview_22-10-20-skompresowany.md_09',
 'TWOW_07_Rulebook_Main_(36p)_EN_preview_22-10-20-skompresowany.md_10',
 'TWOW_07_Rulebook_Main_(36p)_EN_preview_22-10-20-skompresowany.md_11',
 'TWOW_07_Rulebook_Main_(36p)_EN_preview_22-10-20-skompresowany.md_41',
 'TWOW_07_Rulebook_Main_(36p)_EN_preview_22-10-20-skompresowany.md_42',
 'TWOW_07_Rulebook_Main_(36p)_EN_preview_22-10-20-skompresowany.md_43']

In [10]:
results['documents']

['Dice Place all Gold Tokens and the two sets of Poker Dice near the Game Board. 7 Location Tokens Sort all Location Tokens into 3 piles (according to their Terrain types of: Forest, Mountain, and Water). Shuffle each pile separately and place them in separate facedown piles near the Game Board. Draw 1 token from each pile and place them face up on their corresponding slots on the Game Board. 8 Monsters 1. Sort all Monster Cards into 3 separate piles (based on their Level: I, II, or III) and place them nearby face up, showing Monster’s illustration, Life Pool, and Special Ability. 2. Sort all Monster Tokens into 3 separate piles (based on their Level: I, II, or III). Shuffle each pile separately and place them nearby face down. 3. Draw 3 Tokens from the Level I Stack of Monster Tokens; place them randomly, 1 near each of the face-up Location Tokens on the Game Board, in the Monster section. - 2-Player Game: Draw 2 Tokens from the Level I Stack of Monster Tokens and 1 from the Level II 

In [11]:
# Create the final context
context = ' '.join(results['documents'])

As the LLM of choice, we went for a very small quantized model in order to fit in 12GB VRAM.  
Phi-3-mini-128k-instruct is a good choice, as it also has a huge context window, and it is intruction trained.  
In fact, with such a low VRAM, we cannot use the whole context window.  
If you have trouble fitting into your VRAM, play with the value max_new_tokens=20000.  

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import sentencepiece
import accelerate

bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type='nf4',
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True,
    )

model_name='microsoft/Phi-3-mini-128k-instruct'

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, 
                                            device_map="auto",
                                            quantization_config=bnb_config,
                                            trust_remote_code=True,
                                            )

Meager try of prompt engineering here.  
The <|user|> and <|assistant|> special tokens here are how Phi3 expects input to be formatted.  

In [ ]:
input_text = '''<|user|>
Given the following context and question, answer the question based on the context.
Follow this procedure step by step.
1. From the given context, based on the question, find only the relevant sentences, paragraphs and sections.
2. Based on the previous step, remove all the non relevant context from the given context.
3. Answer ONLY based on the remaining context and nothing else.

Context: ''' + context + '''

Question: ''' + question + '''

Answer:
<|assistant|>
'''
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

outputs = model.generate(input_ids, max_new_tokens=20000)

You are not running the flash-attention implementation, expect numerical differences.


We use a list here, in case we want to have memory (continue conversation) in the future.

In [ ]:
outputs_list = []
outputs_list.append(tokenizer.decode(outputs[0]).replace("<|end|>", ""))

This is the RAG answer.  
Even the setup one is kind of OK.  
There is some information missing, but the setup of this game is very complex  
and some setup information is formatted very badly with nested lists and columns in the pdf rulebook.  
With a human corrected markdown rulebook, and perhaps a slightly larger model, even this question can be answered correctly.  

In [ ]:
outputs_list[-1].split("<|assistant|>")[-1]

' To setup a 2-player game for The Witcher: Old World, follow these steps:\n\n1. Setup the Game Board in the middle of the table.\n\n2. Create the Action Card Deck by shuffling all Action Cards and placing them facedown on the indicated spot near the Game Board.\n\n3. Form the Action Cards Pool by revealing Action Cards until 3 "0-cost" cards are revealed. Place these 3 cards face up on the 3 right-most slots on the Game Board.\n\n4. Shuffle and place the remaining revealed Action Cards on the indicated spot near the Game Board.\n\n5. Place 3 Action Cards from the Action Deck on the three remaining slots on the Game Board.\n\n6. Setup the Potion Deck by shuffling all Potion Cards and placing them face down on their indicated space near the Game Board.\n\n7. Place 1 Attribute Trophy Card for each Attribute (Combat, Defense, Alchemy, and Specialty) on their designated spot on the Game Board.\n\n8. Place the remaining 4 cards back in the box.\n\n9. Determine the Starting Player. The Playe

## The Witcher: Old World

**For The Witcher: Old World, please tell me how to setup a 2-player game.**

 To setup a 2-player game for The Witcher: Old World, follow these steps:
 
 1. Setup the Game Board in the middle of the table.
 2. Create the Action Card Deck by shuffling all Action Cards and placing them facedown on the indicated spot near the Game Board.
 3. Form the Action Cards Pool by revealing Action Cards until 3 "0-cost" cards are revealed. Place these 3 cards face up on the 3 right-most slots on the Game Board.
 4. Shuffle and place the remaining revealed Action Cards on the indicated spot near the Game Board.
 5. Place 3 Action Cards from the Action Deck on the three remaining slots on the Game Board.
 6. Setup the Potion Deck by shuffling all Potion Cards and placing them face down on their indicated space near the Game Board.
 7. Place 1 Attribute Trophy Card for each Attribute (Combat, Defense, Alchemy, and Specialty) on their designated spot on the Game Board.
 8. Place the remaining 4 cards back in the box.
 9. Determine the Starting Player. The Player who most recently read a Witcher book is the Starting Player; alternatively, you can determine this randomly.
 10. Each Player takes an Action and Fight Help Card.
 11. The Starting Player chooses the Terrain type for the Level II Monster
 12. Setup for a 2-Player Game is complete.  

**In The Witcher: Old World, can I play poker and fight a player at the same turn?**

Yes, in the same turn, you can choose to play Dice Poker with another Witcher and also decide to Fight with a Witcher. However, you cannot Fight with a Witcher you played Dice Poker with in the same turn.  

## Dune: Imperium - Uprising

**In Dune: Imperium - Uprising, please explain in detail how spies and observation posts work.**

In the game, spies and observation posts are strategic elements that allow players to gather intelligence and potentially infiltrate other players' territories. Here's a detailed explanation:

1. **Observation Posts**: These are various board spaces across the game board. Each observation post is connected to one or more board spaces. Whenever the Spy icon appears on a card or board space, a player may place a Spy from their supply on an unoccupied observation post on the board.

2. **Placement of Spies**: A player can only place a Spy on an observation post if they have a Spy in their supply. If a player has no Spies in their supply when they need to place one, they may first recall one of their Spies for no effect.

3. **Recall of Spies**: Spies can be recalled from an observation post without using a card. There are two effects of recalling a Spy:
   - **Infiltrate**: If a player wishes to send an Agent to a board space occupied by another player, they may recall their own Spy from a connected observation post to ignore the other player’s Agent and send their Agent to that same board space.
   - **Gather Intelligence**: Whenever a player sends an Agent to a board space, they may recall their own Spy from a connected observation post to draw a card. This can only be done immediately after placing the Agent.

4. **Spy Agent Icon**: This allows a player to send an Agent to a board space connected to an observation post where they currently have a Spy. However, the Spy is not recalled for this purpose.

In summary, spies and observation posts add a layer of strategy to the game, allowing players to gather intelligence and potentially disrupt other players' plans. They are an integral part of the gameplay and can significantly influence the outcome of the game.  

In [20]:
input_text = '''<|user|>
''' + questions[selected_question] + '''
<|assistant|>
'''
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

outputs = model.generate(input_ids, max_new_tokens=20000)

In [21]:
outputs_list.append(tokenizer.decode(outputs[0]).replace("<|end|>", ""))

This is the answer without using RAG.  
As you can see, the answers are complete hallucinations.  

In [22]:
outputs_list[-1].split("<|assistant|>")[-1]

' To set up a 2-player game for "The Witcher: Old World", follow these steps:\n\n\n1. **Purchase the Game**: Ensure you have two copies of "The Witcher: Old World" for each player.\n\n\n2. **Install the Game**: Install the game on your respective systems. If you\'re playing on different platforms (PC, PlayStation, Xbox, etc.), you\'ll need to install the game on each of your systems.\n\n\n3. **Connect Online**: If the game supports online play, you\'ll need to connect to the same online gaming service (like PlayStation Network or Xbox Live) to play together.\n\n\n4. **Create Accounts**: Create separate accounts for each player if required by the game.\n\n\n5. **Start a Game**: Launch the game on both systems. You can either start a new game or load an existing one.\n\n\n6. **Join the Game**: On the system where you\'re hosting the game, you\'ll need to join the game session. This is usually done through the game\'s main menu or a specific \'Join Game\' option.\n\n\n7. **Play Together**

## The Witcher: Old World

**For The Witcher: Old World, please tell me how to setup a 2-player game.**

To set up a 2-player game for "The Witcher: Old World", follow these steps:

1. **Purchase the Game**: Ensure you have two copies of "The Witcher: Old World" for each player.

2. **Install the Game**: Install the game on your respective systems. If you're playing on different platforms (PC, PlayStation, Xbox, etc.), you'll need to install the game on each of your systems.

3. **Connect Online**: If the game supports online play, you'll need to connect to the same online gaming service (like PlayStation Network or Xbox Live) to play together.

4. **Create Accounts**: Create separate accounts for each player if required by the game.

5. **Start a Game**: Launch the game on both systems. You can either start a new game or load an existing one.

6. **Join the Game**: On the system where you're hosting the game, you'll need to join the game session. This is usually done through the game's main menu or a specific 'Join Game' option.

7. **Play Together**: Once joined, you can play together, with each player controlling their own Geralt of Rivia character.

Remember to check the game's specific instructions or community forums for any additional steps or tips specific to "The Witcher: Old World".  


**In The Witcher: Old World, can I play poker and fight a player at the same turn?**

No, in The Witcher: Old World board game, you cannot perform two actions that require different types of resources in the same turn. Poker is a card game that requires gold, while fighting a player is a combat action that requires a sword.  


## Dune: Imperium - Uprising

**In Dune: Imperium - Uprising, please explain in detail how spies and observation posts work.**

In the "Dune: Imerium - Uprising" board game, spies play a crucial role in gathering information and gaining strategic advantages. Here\'s a detailed guide on how you can use spies effectively:\n\n1. Understanding Spy Mechanics: Spies are represented by the Spy tokens in the game. Each player starts with a certain number of these tokens, depending on their faction. Spies can be used to gather information about opponents\' plans, resources, and positions.\n\n2. Gathering Information: To use a spy, you need to place it on a specific type of space called an Information Space. These spaces are usually marked with a symbol or color indicating their purpose. Information Spaces are found on the game board and are strategically placed to provide information about different aspects of the game.\n\n3. Information Gathering: When you place a spy on an Information Space, it will reveal information about the opponent\'s resources, plans, or positions. The information gained can be used to make informed decisions about your own strategy.\n\n4. Information Usage: The information gathered by spies can be used in various ways. For example, you can use it to plan your moves, predict opponents\' actions, or even sabotage their plans. Information can also be used to gain advantages in resource allocation or to influence the outcome of certain events in the game.\n\n5. Limitations and Risks: Using spies comes with certain limitations and risks. Spies can only be used once per turn, and they cannot be used to directly attack opponents. Additionally, spies can be detected by opponents, which may lead to countermeasures or retaliation.\n\n6. Strategic Placement: To maximize the effectiveness of spies, it\'s important to strategically place them on Information Spaces. Consider the information you need to gather and the opponents you want to monitor. Placing spies in locations that provide valuable information about multiple opponents can be particularly advantageous.\n\n7. Balancing Spy Usage: While spies can provide significant advantages, it\'s important to balance their usage with other gameplay elements. Over-reliance on spies may lead to predictability in your strategy, making it easier for opponents to counter.\n\nIn summary, spies in "Dune: Imerium - Uprising" are a powerful tool for gathering information and gaining strategic advantages. By understanding their mechanics, strategically placing them, and using the information effectively, you can enhance your gameplay and increase your chances of success.

**In Dune: Imperium - Uprising, please explain in detail how spies and observation posts work.**

In the Dune: Imperium - Uprising game, spies and observation posts play a crucial role in gathering information and gaining strategic advantages over opponents.

Spies:
Spies are specialized units that can be deployed by players to infiltrate enemy territories and gather intelligence. They are represented by unique icons on the game board. Here's how they work:\n\n1. Deployment: Players can deploy spies during their turn, but they must first spend a specific resource (e.g., Spice) to do so. The number of spies a player can deploy depends on their level and resources.\n\n2. Movement: Spies can move across the game board, but their movement is limited by their level and the terrain. They can move one space per turn, and they cannot move through impassable terrain like mountains or water.\n\n3. Gathering Information: When a spy enters an enemy territory, they can gather information about the enemy's forces, resources, and plans. This information is represented by cards that are placed in the spy's possession.\n\n4. Sabotage: Spies can also sabotage enemy operations by disrupting their supply lines, damaging their infrastructure, or assassinating key targets. This is done by using special abilities that are revealed when a spy is deployed.\n\nObservation Posts:\nObservation posts are strategically placed on the game board to provide players with information about the state of the game. They are represented by icons that resemble telescopes or binoculars. Here's how they work:\n\n1. Placement: Players can place observation posts on any unoccupied space on the game board during their turn. Each observation post requires a specific resource (e.g., Spice) to be placed.\n\n2. Information Gathering: Once placed, an observation post provides players with information about the state of the game. This includes the number of spies each player has, the resources each player has, and the number of territories each player controls.\n\n3. Upgrades: Players can upgrade their observation posts to gain additional information. Upgrades require a specific resource (e.g., Spice) and can be done during a player's turn.\n\n4. Limitations: Observation posts have limitations on the amount of information they can provide. For example, they may only provide information about the state of the game up to a certain point in time.\n\nIn summary, spies and observation posts are essential components of the Dune: Imperium - Uprising game. Spies provide players with valuable intelligence and the ability to sabotage enemy operations, while observation posts provide players with information about the state of the game. Both units require resources to deploy and maintain, and they can be upgraded to provide additional benefits.